In [233]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
    <script> code_show=true; function code_toggle() {if (code_show){$('div.input').hide();} else {$('div.input').show();} code_show = !code_show} $( document ).ready(code_toggle);</script>
    <form action="javascript:code_toggle()"><input type="submit" value="Hide/show all code."></form>
''')

# Sydney Liveability Analysis

## Group - 5  F10Adv-01

In [234]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

In [235]:
sa2_data = gpd.read_file("SA2_2016_AUST/SA2_2016_AUST.shp")

In [236]:
break_and_enter = gpd.read_file("break_and_enter/BreakEnterDwelling_JanToDec2021.shp")

In [237]:
school_catchments_future = gpd.read_file("school_catchments/catchments_future.shp")

In [238]:
school_catchments_primary = gpd.read_file("school_catchments/catchments_primary.shp")

In [239]:
school_catchments_secondary = gpd.read_file("school_catchments/catchments_secondary.shp")

In [240]:
neighbourhoods = pd.read_csv('Neighbourhoods.csv')

In [241]:
BusinessStats = pd.read_csv('BusinessStats.csv')

In [242]:
Playgrounds_json = json.load(open('Playgrounds.geojson'))

Playgrounds_DataFrame = pd.DataFrame(Playgrounds_json["features"])
Playgrounds_pandas_data=[]
for index, row in Playgrounds_DataFrame.iterrows():
    append_row = [row["geometry"]["coordinates"][0],row["geometry"]["coordinates"][1],row["properties"]["Name"]]
    Playgrounds_pandas_data.append(append_row)
    
Playgrounds = pd.DataFrame(Playgrounds_pandas_data, columns = ['X', 'Y','Name'])
Playgrounds['geom'] = gpd.points_from_xy(Playgrounds.X, Playgrounds.Y)  # creating the geometry column
Playgrounds = Playgrounds.drop(columns=['X', 'Y'])  # removing the old latitude/longitude fields

In [243]:
Lights_json = json.load(open('Lights.geojson'))
Lights_DataFrame = pd.DataFrame(Lights_json["features"])
Lights_pandas_data=[]
for index, row in Lights_DataFrame.iterrows():
    append_row = [row["geometry"]["coordinates"][0],row["geometry"]["coordinates"][1],row["properties"]["Location"]]
    Lights_pandas_data.append(append_row)
    
Lights = pd.DataFrame(Lights_pandas_data, columns = ['X', 'Y','Location'])
Lights['geom'] = gpd.points_from_xy(Lights.X, Lights.Y)  # creating the geometry column
Lights = Lights.drop(columns=['X', 'Y'])  # removing the old latitude/longitude fields

In [244]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [245]:
db, conn = pgconnect(credentials)

Connected successfully.


In [246]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.0 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [247]:
srid = 4326
Playgrounds['geom'] = Playgrounds['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
Lights['geom'] = Lights['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [248]:
def create_wkt_element(geom, srid):
    if geom!=None :
        if geom.geom_type == 'Polygon':
            geom = MultiPolygon([geom])
        return WKTElement(geom.wkt, srid)

sa2_data_og = sa2_data.copy()  # creating a copy of the original for later
sa2_data['geom'] = sa2_data['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
sa2_data = sa2_data.drop(columns="geometry")  # deleting the old copy

break_and_enter_og = break_and_enter.copy()  # creating a copy of the original for later
break_and_enter['geom'] = break_and_enter['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
break_and_enter = break_and_enter.drop(columns="geometry")  # deleting the old copy

school_catchments_future_og = school_catchments_future.copy()  # creating a copy of the original for later
school_catchments_future['geom'] = school_catchments_future['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
school_catchments_future = school_catchments_future.drop(columns="geometry")  # deleting the old copy

school_catchments_primary_og = school_catchments_primary.copy()  # creating a copy of the original for later
school_catchments_primary['geom'] = school_catchments_primary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
school_catchments_primary = school_catchments_primary.drop(columns="geometry")  # deleting the old copy

school_catchments_secondary_og = school_catchments_secondary.copy()  # creating a copy of the original for later
school_catchments_secondary['geom'] = school_catchments_secondary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
school_catchments_secondary = school_catchments_secondary.drop(columns="geometry")  # deleting the old copy

In [257]:
print(type(sa2_data))
print(type(BusinessStats))
BusinessStats

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'pandas.core.frame.DataFrame'>


,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101021007,Braidwood,629,26,27,280,11,0,35
1,101021008,Karabar,326,7,10,8,11,0,43
2,101021009,Queanbeyan,724,52,47,11,56,3,77
3,101021010,Queanbeyan - East,580,16,23,4,12,0,57
4,101021011,Queanbeyan Region,1642,39,63,292,34,7,81
...,...,...,...,...,...,...,...,...,...
2296,901011001,Christmas Island,0,0,0,0,0,0,0
2297,901021002,Cocos (Keeling) Islands,7,3,0,0,0,0,0
2298,901031003,Jervis Bay,6,0,3,0,0,0,0
2299,901041004,Norfolk Island,0,0,0,0,0,0,0


In [275]:
# conn.execute("""
# CREATE SCHEMA IF NOT EXISTS Sydney_Liveability_Analysis;
# SET search_path TO Sydney_Liveability_Analysis;
# """)

# sa2_data
conn.execute("""
DROP TABLE IF EXISTS sa2_data;
CREATE TABLE sa2_data (
    "SA2_MAIN16" INTEGER, 
    "SA2_5DIG16" INTEGER, 
    "SA2_NAME16" VARCHAR(80),
    "SA3_CODE16" INTEGER,
    "SA3_NAME16" VARCHAR(80),
    "SA4_CODE16" INTEGER,
    "SA4_NAME16" VARCHAR(80),
    "GCC_CODE16" VARCHAR(80),
    "GCC_NAME16" VARCHAR(80),
    "STE_CODE16" INTEGER,
    "STE_NAME16" VARCHAR(80),	
    "AREASQKM16" NUMERIC,
    "geom" GEOMETRY(MULTIPOLYGON,4326)
);"""
)

# neighbourhood
conn.execute("""
DROP TABLE IF EXISTS neighbourhood;
CREATE TABLE neighbourhood (
    "Unnamed: 0" INTEGER,
    "area_id" INTEGER,
    "area_name" VARCHAR(80),
    "land_area" NUMERIC,
    "population" VARCHAR(80),
    "number_of_dwellings" VARCHAR(80),
    "number_of_businesses" INTEGER,
    "median_annual_household_income" NUMERIC,
    "avg_monthly_rent" INTEGER,
    "0-4" INTEGER,
    "5-9" INTEGER,
    "10-14" INTEGER,
    "15-19" INTEGER 
);"""
)

# break and enter
conn.execute("""
DROP TABLE IF EXISTS break_and_enter;
CREATE TABLE break_and_enter (
    "OBJECTID" INTEGER, 
    "Contour" NUMERIC, 
    "Density" VARCHAR(80), 
    "ORIG_FID" INTEGER, 
    "Shape_Leng" NUMERIC,
    "Shape_Area" NUMERIC,
    "geom" GEOMETRY(MULTIPOLYGON,4326)
);"""
)

# school_catchments_future
conn.execute("""
DROP TABLE IF EXISTS school_catchments_future;
CREATE TABLE school_catchments_future (
    "USE_ID" INTEGER, 
    "CATCH_TYPE" VARCHAR(80), 
    "USE_DESC" VARCHAR(80), 
    "ADD_DATE" INTEGER, 
    "KINDERGART" INTEGER,
    "YEAR1" INTEGER,
    "YEAR2" INTEGER,
    "YEAR3" INTEGER,
    "YEAR4" INTEGER,
    "YEAR5" INTEGER,
    "YEAR6" INTEGER,
    "YEAR7" INTEGER,
    "YEAR8" INTEGER,
    "YEAR9" INTEGER,
    "YEAR10" INTEGER,
    "YEAR11" INTEGER,
    "YEAR12" INTEGER,
    "geom" GEOMETRY(MULTIPOLYGON,4326)
    
);"""
)

# school_catchments_primary
conn.execute("""
DROP TABLE IF EXISTS school_catchments_primary;
CREATE TABLE school_catchments_primary (
    "USE_ID" INTEGER, 
    "CATCH_TYPE" VARCHAR(80), 
    "USE_DESC" VARCHAR(80), 
    "ADD_DATE" INTEGER, 
    "KINDERGART" VARCHAR(1),
    "YEAR1" VARCHAR(1),
    "YEAR2" VARCHAR(1),
    "YEAR3" VARCHAR(1),
    "YEAR4" VARCHAR(1),
    "YEAR5" VARCHAR(1),
    "YEAR6" VARCHAR(1),
    "YEAR7" VARCHAR(1),
    "YEAR8" VARCHAR(1),
    "YEAR9" VARCHAR(1),
    "YEAR10" VARCHAR(1),
    "YEAR11" VARCHAR(1),
    "YEAR12" VARCHAR(1),
    "PRIORITY" VARCHAR(80),
    "geom" GEOMETRY(MULTIPOLYGON,4326)
    
);"""
)

# school_catchments_secondary
conn.execute("""
DROP TABLE IF EXISTS school_catchments_secondary;
CREATE TABLE school_catchments_secondary (
    "USE_ID" INTEGER, 
    "CATCH_TYPE" VARCHAR(80), 
    "USE_DESC" VARCHAR(80), 
    "ADD_DATE" INTEGER, 
    "KINDERGART" VARCHAR(1),
    "YEAR1" VARCHAR(1),
    "YEAR2" VARCHAR(1),
    "YEAR3" VARCHAR(1),
    "YEAR4" VARCHAR(1),
    "YEAR5" VARCHAR(1),
    "YEAR6" VARCHAR(1),
    "YEAR7" VARCHAR(1),
    "YEAR8" VARCHAR(1),
    "YEAR9" VARCHAR(1),
    "YEAR10" VARCHAR(1),
    "YEAR11" VARCHAR(1),
    "YEAR12" VARCHAR(1),
    "PRIORITY" VARCHAR(80),
    "geom" GEOMETRY(MULTIPOLYGON,4326)
    
);"""
)


# BusinessStats
conn.execute("""
DROP TABLE IF EXISTS businessstats;
CREATE TABLE businessstats (
    "area_id" INTEGER,
    "area_name" VARCHAR(100),
    "number_of_businesses" INTEGER,
    "accommodation_and_food_services" INTEGER,
    "retail_trade" INTEGER,
    "agriculture_forestry_and_fishing" INTEGER,
    "health_care_and_social_assistance" INTEGER,
    "public_administration_and_safety" INTEGER,
    "transport_postal_and_warehousing" INTEGER
);"""
)

# Playgrounds
conn.execute("""
DROP TABLE IF EXISTS playgrounds;
CREATE TABLE playgrounds (
   "Name" VARCHAR(200),
   geom GEOMETRY(POINT, 4326)
);"""
)

# Lights
conn.execute("""
DROP TABLE IF EXISTS lights;
CREATE TABLE lights (
   "Location" VARCHAR(200),
   geom GEOMETRY(POINT, 4326)
);"""
)

In [278]:
sa2_data.to_sql('sa2_data', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
# query(conn,"select * from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='sa2_data'")
# query(conn, "select * from sa2_data")

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geom
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,0106000020E6100000010000000103000000010000005F...
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,0106000020E6100000010000000103000000010000002C...
2,101031016,11016,Jindabyne - Berridale,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3939.5484,0106000020E6100000010000000103000000010000005F...
3,101041017,11017,Batemans Bay,10104,South Coast,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,63.7074,0106000020E61000000100000001030000000100000036...
4,101041018,11018,Batemans Bay - South,10104,South Coast,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,30.5948,0106000020E6100000070000000103000000010000001C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,0106000020E6100000380000000103000000010000000E...
4616,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,0106000020E61000000300000001030000000100000017...
4617,901041004,91004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,38.6509,0106000020E61000001F0000000103000000010000000B...
4618,997979799,99799,Migratory - Offshore - Shipping (OT),99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,0.0000,None


In [252]:
neighbourhoods.to_sql('neighbourhood', conn, if_exists='append', index=False)
# query(conn, "select * from neighbourhood")

In [253]:
break_and_enter.to_sql("break_and_enter", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
# query(conn, "select * from break_and_enter")

In [254]:
school_catchments_future.to_sql("school_catchments_future", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
# query(conn, "select * from school_catchments_future")

In [255]:
school_catchments_primary.to_sql("school_catchments_primary", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
# query(conn, "select * from school_catchments_primary")

In [256]:
school_catchments_secondary.to_sql("school_catchments_secondary", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
# query(conn, "select * from school_catchments_secondary")

In [266]:
BusinessStats.to_sql('businessstats', conn, if_exists='append', index=False)
query(conn, "select * from businessstats")

,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101021007,Braidwood,629,26,27,280,11,0,35
1,101021008,Karabar,326,7,10,8,11,0,43
2,101021009,Queanbeyan,724,52,47,11,56,3,77
3,101021010,Queanbeyan - East,580,16,23,4,12,0,57
4,101021011,Queanbeyan Region,1642,39,63,292,34,7,81
...,...,...,...,...,...,...,...,...,...
2296,901011001,Christmas Island,0,0,0,0,0,0,0
2297,901021002,Cocos (Keeling) Islands,7,3,0,0,0,0,0
2298,901031003,Jervis Bay,6,0,3,0,0,0,0
2299,901041004,Norfolk Island,0,0,0,0,0,0,0


In [272]:
Lights.to_sql('lights', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
query(conn, "select * from lights")

,Location,geom
0,"Mounted on pole, illuminates street near 228 K...",0101000020E6100000548B6352D3E562403862DB3FA2F2...
1,"Mounted on pole, illuminates street near 1B Wh...",0101000020E61000006D2593A4D2E56240C01E2455A5F2...
2,"Mounted on ground, is an uplight near 1 Chifle...",0101000020E6100000E0683561C1E66240206579C1E1EE...
3,"Mounted on ground, is an uplight near 1 Chifle...",0101000020E61000000E20C1FCC1E6624098C545FADCEE...
4,"Mounted on ground, is an uplight near 1 Chifle...",0101000020E6100000150D138CC1E66240E8FFE183DEEE...
...,...,...
8428,"O'Brien Lane Reserve, Darlinghurst",0101000020E61000006561AC62F6E66240AFB8BEF629F0...
8429,Minogue Crescent side of Canal. Supplied from ...,0101000020E610000088BA5C85A3E5624028F67B88E4F0...
8430,Minogue Crescent side of Canal. Supplied from ...,0101000020E6100000543A9FD4A3E56240B769FCEDEBF0...
8431,Minogue Crescent side of Canal. Supplied from ...,0101000020E61000004C3BD323A4E562401885605FEFF0...


In [276]:
Playgrounds.to_sql('playgrounds', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
query(conn, "select * from playgrounds")

,Name,geom
0,McKell Playground B,0101000020E61000006B9083994BE662401116419B28F3...
1,Minogue Crescent Reserve Playground,0101000020E61000006E221BE4A8E56240BC32B4D4B0F0...
2,May Pitt Playground,0101000020E6100000EB9B6535D0E56240E2D30CDE23F1...
3,Clyne Reserve Playground,0101000020E61000002CCBB99A74E6624085C4981BB0ED...
4,Shannon Reserve Playground,0101000020E6100000FBCD040EDBE662405B122F7C61F1...
...,...,...
146,Redfern Park Playground C,0101000020E61000006013D8149DE6624048B2744394F2...
147,Frances Newton Playground,0101000020E61000009E545B07F1E66240F850204547F0...
148,Turruwul Park Playground,0101000020E6100000835784E18BE66240C840BA8793F5...
149,Waterloo Park & Mount Carmel Fitness Station,0101000020E6100000F81F081E8BE662408559CD2427F3...


In [135]:
# query(conn,"select * from INFORMATION_SCHEMA.SCHEMATA")

,catalog_name,schema_name,schema_owner,default_character_set_catalog,default_character_set_schema,default_character_set_name,sql_path
0,y22s1d2x01_prag9782,pg_catalog,postgres,None,None,None,None
1,y22s1d2x01_prag9782,public,postgres,None,None,None,None
2,y22s1d2x01_prag9782,information_schema,postgres,None,None,None,None
3,y22s1d2x01_prag9782,nswfuel,y22s1d2x01_prag9782,None,None,None,None
4,y22s1d2x01_prag9782,carhire,y22s1d2x01_prag9782,None,None,None,None
5,y22s1d2x01_prag9782,prices,y22s1d2x01_prag9782,None,None,None,None


In [136]:
conn.close()
db.dispose()